In [1]:
import warnings
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd

warnings.filterwarnings("ignore")

In [2]:
cred = credentials.Certificate("slts-8e29d-firebase-adminsdk-i6d1u-d3ea7654ac.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [3]:
registrationData = db.collection('registrationData').stream()
data = []
for doc in registrationData:
    data.append(doc.to_dict())

df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
df.columns

In [6]:
df['event1'] = df['registeredEvents'].apply(lambda x: x[0] if len(x) > 0 else "")
df['event2'] = df['registeredEvents'].apply(lambda x: x[1] if len(x) > 1 else "")

In [ ]:
new_df = df[['studentId', 'studentFullName', 'studentGroup', 'district', 'samithiName', 'gender', 'event1', 'event2', 'score', 'comment',]]
new_df

In [8]:
data = new_df.to_dict(orient='records')

In [9]:
def totalScore(score, event):
    if type(score) is not dict:
        return 0
    if type(event) is not str:
        return 0
    if event not in score:
        return 0
    
    total = 0
    for _, value in score[event].items():
        if value:
            for _, v in value.items():
                if str(v).isdigit():
                    total += int(v)

    return total

def extractComment(comment, event):
    if type(comment) is not dict:
        return ""
    if type(event) is not str:
        return ""
    if event not in comment:
        return ""
    

    commentString = ""
    for _, value in comment[event].items():
        if value:
            value = value.strip()
            value = value.replace("\n", " ")
            value = value.replace("\r", " ")
            value = value.replace("\t", " ")
            value = value.replace(",", " ")
            commentString += value + "......."

    return commentString

for d in data:
    d['event1Score'] = totalScore(d['score'], d['event1'])
    d['event2Score'] = totalScore(d['score'], d['event2'])
    d['event1Comment'] = extractComment(d['comment'], d['event1'])
    d['event2Comment'] = extractComment(d['comment'], d['event2'])


In [ ]:
new_df = pd.DataFrame(data)
new_df

In [ ]:
new_df.drop(['score', 'comment'], axis=1, inplace=True)
# Sort by district,
new_df.sort_values(by=['district'], inplace=True)
new_df

In [12]:
new_df.to_csv('scores_comments_slts_2024.csv', index=False)

In [ ]:
df = df[['studentId', 'studentFullName', 'studentGroup', 'district', 'samithiName', 'gender', 'event1', 'event2', 'registeredEvents']]
df

In [ ]:
unique_events = df['registeredEvents'].apply(lambda x: x[0] if len(x) > 0 else "").unique()
unique_events = unique_events.tolist()
unique_events.extend(df['registeredEvents'].apply(lambda x: x[1] if len(x) > 1 else "").unique().tolist())
unique_events = list(set(unique_events))
unique_events.remove("")
unique_events

In [ ]:
unique_districts = df['district'].unique()
unique_districts = unique_districts.tolist()
unique_districts

In [16]:
event_stat = {}
for event in unique_events:
    event_stat[event] = {
        "Male": 0,
        "Female": 0,
        "Total": 0,
    }
    for district in unique_districts:
        event_stat[event][district] = 0
# event_stat

In [ ]:
df

In [ ]:
for _, row in df.iterrows():
    try:
        if row['event1'] != "":
            event1 = row['event1']
            event_stat[event1][row['gender']] += 1
            event_stat[event1][row['district']] += 1
            event_stat[event1]['Total'] += 1

        if row['event2'] != "":
            event2 = row['event2']
            event_stat[event2][row['gender']] += 1
            event_stat[event2][row['district']] += 1
            event_stat[event2]['Total'] += 1
        
    except Exception as e:
        print(e)

# Find count of unique district in each event
for event in unique_events:
    event_stat[event]['uniqueDistricts'] = 0
    event_stat[event]['districtList'] = ""
    for district in unique_districts:
        if event_stat[event][district] > 0:
            event_stat[event]['uniqueDistricts'] += 1
            event_stat[event]['districtList'] += district + "..."
    


event_stat_df = pd.DataFrame(event_stat)
# invert the dataframe
event_stat_df = event_stat_df.T
event_stat_df.index.name = 'Event'
event_stat_df = event_stat_df[['Male', 'Female', 'Total', 'uniqueDistricts', 'districtList']]
event_stat_df

In [19]:
event_stat_df.to_csv('event_stat_slts_2024.csv')

In [ ]:
# Overall stats
overall_stats = {}

for district in unique_districts:
    overall_stats[district] = {
        'Male': 0,
        'Female': 0,
        'Total': 0,
    }

for _, row in df.iterrows():
    try:
        overall_stats[row['district']]['Total'] += 1
        overall_stats[row['district']][row['gender']] += 1
    except Exception as e:
        print(e)

overall_stats_df = pd.DataFrame(overall_stats)
overall_stats_df = overall_stats_df.T
overall_stats_df.index.name = 'District'
overall_stats_df

In [ ]:
overall_stats_df.to_csv('overall_stats_slts_2024.csv')